### Introduction

The purpose of this workbook is to use the Simple Salesforce python package to extract all completed Tasks with a subject containing the word 'call', then total these up for each Lead/Contact and populate a custom field *Number_Of_Calls__c* on each Lead/Contact record.

Schema details for the Task object in Salesforce can be found here:

https://developer.salesforce.com/docs/atlas.en-us.api.meta/api/sforce_api_erd_activities.htm

Simple Salesforce documentation can be found here:

https://pypi.python.org/pypi/simple-salesforce

#### imports etc.

In [19]:
import pandas as pd
from simple_salesforce import Salesforce

# Enter the relevant credentials for your Salesforce org.
sf = Salesforce(password='xxxxxxxxxxxxx', \
                username='xxxxxxxxxxxxx', \
                organizationId='xxxxxxxxxxxxx',\
                security_token='xxxxxxxxxxxxx')


#### First step is to run a query on all completed tasks with subject containing the word 'call'.

In [20]:
# Salesforce SOQL query for all completed Tasks with 'call' in the Subject:
query= """
SELECT Id, WhoId, WhatId, OwnerId, AccountId, Subject, Status
FROM Task
WHERE Subject LIKE '%call%' AND Status = 'Completed'
"""
tasks_df = pd.DataFrame(sf.query_all(query)['records'])  # Create dataframe from Salesforce query.
tasks_df.drop('attributes',axis=1,inplace=True)  # Drop 'attributes' column which is not required.

# See how many records there are (to make sure data has been produced).
print 'Total number of completed task \'call\' records = ',tasks_df.Id.count()

Total number of completed task 'call' records =  1270


In [21]:
tasks_df.head() # Have a look at the dataframe

,AccountId,Id,OwnerId,Status,Subject,WhatId,WhoId
0,None,00Tb000000b7Wq3EAE,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008EImqEAG
1,None,00Tb000000b7kvOEAQ,005b0000001ehRRAAY,Completed,Call,None,00Qb00000077nbyEAA
2,None,00Tb000000b7rzPEAQ,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008DtqwEAC
3,None,00Tb000000b8GgrEAE,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008F5PAEA0
4,None,00Tb000000b8HNEEA2,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008EFmKEAW


#### Next we need the Lead/Contact Ids to identify which of the *WhoId* values are Leads and which are Contacts.

In [22]:
query= "SELECT Id FROM Lead" # Salesforce SOQL query for all Lead Ids
leads_df = pd.DataFrame(sf.query_all(query)['records'])  # Create dataframe from Salesforce query.
leads_df = leads_df.rename(columns={'Id':'WhoId'})  # Change Id column name so it can be merged with Tasks dataframe.
leads_df.drop('attributes',axis=1,inplace=True)  # Drop 'attributes' column as not required.
leads_df['Is_Lead']=1  # Add column to merge into Tasks dataframe which identifes the record as a Lead.

query= "SELECT Id FROM Contact" # Salesforce SOQL query for all Contact Ids
contacts_df = pd.DataFrame(sf.query_all(query)['records'])  # Create dataframe from Salesforce query.
contacts_df = contacts_df.rename(columns={'Id':'WhoId'})  # Change Id column name so it can be merged with Tasks dataframe.
contacts_df.drop('attributes',axis=1,inplace=True)  # Drop 'attributes' column as not required.
contacts_df['Is_Contact']=1  # Add column to merge into Tasks dataframe which identifes the record as a Contact.

# See how many records there are of each type (make sure data has been produced)
print 'Total number of Lead records = ',leads_df.WhoId.count()
print 'Total number of Contact records = ',contacts_df.WhoId.count()

Total number of Lead records =  19935
Total number of Contact records =  4850


In [23]:
# Merge the 'Is_Lead'/'Is_Contact' fields into Tasks dataframe.
tasks_df = pd.merge(tasks_df,leads_df,how='left',on='WhoId')
tasks_df = pd.merge(tasks_df,contacts_df,how='left',on='WhoId')

# Drop rows where the Task was not related to a Lead or Contact.
tasks_df.dropna(axis=0,how='all',subset=('Is_Contact','Is_Lead'),inplace=True)

tasks_df.head() # Have a look at the updated dataframe

,AccountId,Id,OwnerId,Status,Subject,WhatId,WhoId,Is_Lead,Is_Contact
0,None,00Tb000000b7Wq3EAE,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008EImqEAG,1.0,NaN
1,None,00Tb000000b7kvOEAQ,005b0000001ehRRAAY,Completed,Call,None,00Qb00000077nbyEAA,1.0,NaN
2,None,00Tb000000b7rzPEAQ,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008DtqwEAC,1.0,NaN
3,None,00Tb000000b8GgrEAE,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008F5PAEA0,1.0,NaN
4,None,00Tb000000b8HNEEA2,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008EFmKEAW,1.0,NaN


#### Now we need to total up the number of completed calls for each Lead / Contact.

In [24]:
# Add new 'Call_Count' column which adds a total count of each 'WhoId'
tasks_df['Call_Count'] = tasks_df.groupby('WhoId')['WhoId'].transform("count")
tasks_df.head() # Have a look at the updated dataframe

,AccountId,Id,OwnerId,Status,Subject,WhatId,WhoId,Is_Lead,Is_Contact,Call_Count
0,None,00Tb000000b7Wq3EAE,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008EImqEAG,1.0,NaN,1
1,None,00Tb000000b7kvOEAQ,005b0000001ehRRAAY,Completed,Call,None,00Qb00000077nbyEAA,1.0,NaN,1
2,None,00Tb000000b7rzPEAQ,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008DtqwEAC,1.0,NaN,2
3,None,00Tb000000b8GgrEAE,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008F5PAEA0,1.0,NaN,1
4,None,00Tb000000b8HNEEA2,005b0000001ehRRAAY,Completed,Call,None,00Qb0000008EFmKEAW,1.0,NaN,2


#### Now that we have the total number of completed calls for every Lead and Contact the last step is to update the *Number_Of_Calls__c* field in the records in Salesforce.

In [25]:
# Create Leads dataframe with data to push to Salesforce
leads_to_update = tasks_df[tasks_df['Is_Lead']==1][['WhoId' , 'Call_Count']] # Create a Lead dataframe
leads_to_update.drop_duplicates(inplace=True) # Remove duplicates (because 1 row for every call)
leads_to_update.reset_index(inplace=True) # Reset index

# Create Contacts dataframe with data to push to Salesforce
contacts_to_update = tasks_df[tasks_df['Is_Contact']==1][['WhoId' , 'Call_Count']] # Create a Contact dataframe
contacts_to_update.drop_duplicates(inplace=True) # Remove duplicates (because 1 row for every call)
contacts_to_update.reset_index(inplace=True) # Reset index

In [ ]:
# Update 'Number_Of_Calls__c' field in Leads
for i in range(0,int(leads_to_update['WhoId'].count()),1):
    record_id = leads_to_update['WhoId'][i]
    call_count = int(leads_to_update['Call_Count'][i])
    sf.Lead.update(record_id,{'Number_Of_Calls__c': call_count})

In [ ]:
# Update 'Number_Of_Calls__c' field in Contacts
for i in range(0,int(contacts_to_update['WhoId'].count()),1):
    record_id = contacts_to_update['WhoId'][i]
    call_count = int(contacts_to_update['Call_Count'][i])
    sf.Contact.update(record_id,{'Number_Of_Calls__c': call_count})